In [1]:
import dagshub
dagshub.init(repo_owner='fienme', repo_name='testmlflow', mlflow=True)
import mlflow
mlflow.get_tracking_uri()


Accessing as fienme

Initialized MLflow to track repo "fienme/testmlflow"

Repository fienme/testmlflow initialized!

'https://dagshub.com/fienme/testmlflow.mlflow'

In [2]:
experiment_id = mlflow.create_experiment("wine_quality_xgboost_v12")
mlflow.get_tracking_uri()

RestException: BAD_REQUEST: Response: {'error_code': 'BAD_REQUEST'}

In [ ]:
import logging
import warnings


# Wine Quality Sample
def train(it_n_estimators, it_learning_rate, it_max_depth):
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from xgboost import XGBRegressor
    from urllib.parse import urlparse
    import json

    import mlflow
    import mlflow.sklearn
    import mlflow.xgboost
    from mlflow.models import infer_signature

    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

  
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    warnings.filterwarnings("ignore")
    np.random.seed(40)


    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            f"Unable to download training & test CSV, check your internet connection. Error: {e}"
        )


    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data, test_size=0.25, random_state=40)


    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no n_estimators is provided
    n_estimators = 100 if int(it_n_estimators) is None else int(it_n_estimators)

    # Set default values if no learning_rate is provided
    learning_rate = 0.1 if float(it_learning_rate) is None else float(it_learning_rate)

    # Set default values if no max_depht is provided
    max_depth = 5 if int(it_max_depth) is None else int(it_max_depth)

    # logging list with features in json file
    target_column = "quality"
    features = [col for col in data.columns if col != target_column]
    print("Feature names:", features)

    with open("features.json", "w") as f:
        json.dump(features, f)

    # Useful for multiple runs (only doing one run in this sample notebook)
    with mlflow.start_run(experiment_id):
        # Execute ElasticNet
        lr = XGBRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=40
        )
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print(f"XGBoost Regressor (n_estimators={n_estimators:f}, learning_rate={learning_rate:f}, max_depth={max_depth:f}):")
        print(f"  RMSE: {rmse}")
        print(f"  MAE: {mae}")
        print(f"  R2: {r2}")

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.log_param("data_set", "wine_quality_v.2.0.0")

        mlflow.xgboost.log_model(lr, "xgboost_model", registered_model_name="xgboost_v.2.0.0_dataset_v.2.0.0")

        mlflow.data.log_input(mlflow.data.from_pandas(data), context="training")

        mlflow.log_artifact()
        mlflow.log_artifact("features.json")

        # For remote server only (Dagshub)
        remote_server_uri = "https://dagshub.com/fienme/testmlflow.mlflow"
        mlflow.set_tracking_uri(remote_server_uri)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


train(200, 0.2, 10)

In [ ]:
train(150, 0.01, 2)

In [ ]:
train(200, 0.2, 10)

In [ ]:
train(100, 0.1, 5)